In [ ]:
import numpy as np
import tensorflow as tf 
from keras.utils import np_utils
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
current_path = os.getcwd()
a = current_path + '\\frames.csv'

In [ ]:
df = pd.read_csv(a)
df.rename(columns={'Unnamed: 0': 'index',
                   '0': 'image_path'},
          inplace=True)

In [ ]:
updated_path = []
for i in df['image_path']:
    updated_path.append(i.split('\\OneDrive')[0] + i.split('\\OneDrive')[1].replace('\\Sem2\\Big Data', ''))
    
df['image_path'] = updated_path

In [ ]:
ids = []
for i in df['image_path']:
    ids.append(i.split('\\')[-1].split('-')[0])

df['id'] = ids
df['id'] = df['id'].astype(int)

In [ ]:
model = InceptionResNetV2(weights="imagenet", include_top=True, input_shape=(299, 299, 3))
out = model.layers[-2].output
model_final = Model(inputs=model.input, outputs=out)
model_final.summary()

In [ ]:
def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (299, 299))
    return img


In [ ]:
errors = []
for i in range(1,6478):
    try:
        n_path = os.getcwd() + '\\features_inceptionresnet\\' + str(i) + '.npy'
        
        image_list = list(df['image_path'][df['id'] == i])
        print(i, 'is running')
        if len(image_list) == 80:
            images = np.zeros((len(image_list), 299, 299, 3))
            for j in range(len(image_list)):
                img = load_image(image_list[j])
                images[j] = img
            images = np.array(images)
            feats = model_final.predict(images, batch_size=128)
            
            img_feats = np.array(feats)
            np.save(n_path,img_feats)
            print(i, 'is done')
    except:
        print('Error in: ', i)
        errors.append(i)

In [ ]:
e_df = pd.DataFrame(errors)
e_df.to_csv('errors.csv')